# Tools 工具

## 创建工具

In [2]:
from langchain.chat_models import init_chat_model
# 基本工具定义

# 最简单方法是使用 @tool 装饰器

# 类型提示是必需的 ，因为它们定义了工具的输入模式。文档字符串应简明扼要，以帮助模型理解工具的目的。

from langchain.tools import tool


@tool
def search_database(query: str, limit: int = 10) -> str:
  """Search the customer database for records matching the query.

  Args:
      query: Search terms to look for
      limit: Maximum number of results to return
  """
  return f"Found {limit} results for '{query}'"

### 自定义工具属性

In [ ]:
# 自定义工具名称

@tool("web_search")  # Custom name
def search(query: str) -> str:
  """Search the web for information."""
  return f"Results for: {query}"


print(search.name)  # web_search

In [ ]:
# 自定义工具描述

@tool("calculator", description="Performs arithmetic calculations. Use this for any math problems.")
def calc(expression: str) -> str:
  """Evaluate mathematical expressions."""
  return str(eval(expression))

## 高级模式定义

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain.tools import tool


# 定义输入模式（使用 Pydantic）
class WeatherInput(BaseModel):
  """天气查询的输入参数。"""
  location: str = Field(description="城市名称或坐标")
  units: Literal["celsius", "fahrenheit"] = Field(
    default="celsius",
    description="温度单位（摄氏度或华氏度）"
  )
  include_forecast: bool = Field(
    default=False,
    description="是否包含未来 5 天天气预报"
  )


# 定义工具函数，绑定上面的输入模型
@tool(args_schema=WeatherInput)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
  """获取当前天气及可选的未来预报。"""
  temp = 22 if units == "celsius" else 72
  result = f"{location} 当前气温：{temp}° {units[0].upper()}"
  if include_forecast:
    result += "\n未来 5 天：晴朗 ☀️"
  return result


## Accessing Context 访问上下文

为什么这很重要： 当工具能够访问代理状态、运行时上下文和长期记忆时，它们的威力最大。这使得工具能够做出上下文感知的决策、个性化响应，并在对话中保持信息。

工具可以通过 ToolRuntime 参数访问运行时信息，该参数提供：
- 状态 - 在执行过程中流动的可变数据（消息、计数器、自定义字段）
- 上下文 - 不可变的配置，如用户 ID、会话详情或特定于应用程序的配置
- 存储 - 跨对话的持久长期记忆
- 流写入器 - 在工具执行时流式传输自定义更新
- 配置 - 执行的可运行配置
- 工具调用 ID - 当前工具调用的 ID

In [ ]:
# 访问状态

from langchain.tools import tool, ToolRuntime


# Access the current conversation state
@tool
def summarize_conversation(
  runtime: ToolRuntime
) -> str:
  """Summarize the conversation so far."""
  messages = runtime.state["messages"]

  human_msgs = sum(1 for m in messages if m.__class__.__name__ == "HumanMessage")
  ai_msgs = sum(1 for m in messages if m.__class__.__name__ == "AIMessage")
  tool_msgs = sum(1 for m in messages if m.__class__.__name__ == "ToolMessage")

  return f"Conversation has {human_msgs} user messages, {ai_msgs} AI responses, and {tool_msgs} tool results"


# Access custom state fields
@tool
def get_user_preference(
  pref_name: str,
  runtime: ToolRuntime  # ToolRuntime parameter is not visible to the model
) -> str:
  """Get a user preference value."""
  preferences = runtime.state.get("user_preferences", {})
  return preferences.get(pref_name, "Not set")

In [3]:
# 更新状态：

from langgraph.types import Command
from langchain.messages import RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langchain.tools import tool, ToolRuntime


# Update the conversation history by removing all messages
@tool
def clear_conversation() -> Command:
  """Clear the conversation history."""

  return Command(
    update={
      "messages": [RemoveMessage(id=REMOVE_ALL_MESSAGES)],
    }
  )


# Update the user_name in the agent state
@tool
def update_user_name(
  new_name: str,
  runtime: ToolRuntime
) -> Command:
  """Update the user's name."""
  return Command(update={"user_name": new_name})

In [6]:
# 上下文

from dataclasses import dataclass
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime

USER_DATABASE = {
  "user123": {
    "name": "Alice Johnson",
    "account_type": "Premium",
    "balance": 5000,
    "email": "alice@example.com"
  },
  "user456": {
    "name": "Bob Smith",
    "account_type": "Standard",
    "balance": 1200,
    "email": "bob@example.com"
  }
}


@dataclass
class UserContext:
  user_id: str


@tool
def get_account_info(runtime: ToolRuntime[UserContext]) -> str:
  """Get the current user's account information."""
  user_id = runtime.context.user_id

  if user_id in USER_DATABASE:
    user = USER_DATABASE[user_id]
    return f"Account holder: {user['name']}\nType: {user['account_type']}\nBalance: ${user['balance']}"
  return "User not found"


model = ChatOpenAI(model="gpt-4o")
agent = create_agent(
  model,
  tools=[get_account_info],
  context_schema=UserContext,
  system_prompt="You are a financial assistant."
)

result = agent.invoke(
  {"messages": [{"role": "user", "content": "What's my current balance?"}]},
  context=UserContext(user_id="user123")
)

print(result)

{'messages': [HumanMessage(content="What's my current balance?", additional_kwargs={}, response_metadata={}, id='4ffbda6a-6f54-4a36-8da6-ff4778a30919'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 50, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4a331a0222', 'id': 'chatcmpl-CWNP0HsqJYiPay4ZMoOwZ3ogPFFkL', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--4feeed96-de1d-4536-ae26-2e9cdf1b1551-0', tool_calls=[{'name': 'get_account_info', 'args': {}, 'id': 'call_TQRh5DKxTvqV2aSLCTP9e7u2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 50, 'output_tokens': 12, 'total_tokens': 62, 'input_token_details': {'audio': 0, 

### Memory (Store) 内存（存储）

In [7]:
from typing import Any
from langgraph.store.memory import InMemoryStore
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime


# Access memory
@tool
def get_user_info(user_id: str, runtime: ToolRuntime) -> str:
  """Look up user info."""
  store = runtime.store
  user_info = store.get(("users",), user_id)
  return str(user_info.value) if user_info else "Unknown user"


# Update memory
@tool
def save_user_info(user_id: str, user_info: dict[str, Any], runtime: ToolRuntime) -> str:
  """Save user info."""
  store = runtime.store
  store.put(("users",), user_id, user_info)
  return "Successfully saved user info."


store = InMemoryStore()
agent = create_agent(
  model,
  tools=[get_user_info, save_user_info],
  store=store
)

# First session: save user info
agent.invoke({
  "messages": [{"role": "user",
                "content": "Save the following user: userid: abc123, name: Foo, age: 25, email: foo@langchain.dev"}]
})

# Second session: get user info
agent.invoke({
  "messages": [{"role": "user", "content": "Get user info for user with id 'abc123'"}]
})
# Here is the user info for user with ID "abc123":
# - Name: Foo
# - Age: 25
# - Email: foo@langchain.dev

{'messages': [HumanMessage(content="Get user info for user with id 'abc123'", additional_kwargs={}, response_metadata={}, id='4ed6b81c-c623-49d9-8394-00010964e6ef'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 75, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4a331a0222', 'id': 'chatcmpl-CWNTFFHiYZAurEiXSGCeMg8lC0url', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--e52b123f-fffa-43c4-8638-89511b70d007-0', tool_calls=[{'name': 'get_user_info', 'args': {'user_id': 'abc123'}, 'id': 'call_eLCFrMiYFsvx3VJBSG1Dbl7o', 'type': 'tool_call'}], usage_metadata={'input_tokens': 75, 'output_tokens': 18, 'total_tokens': 93, 'inpu

### 流写入器

In [29]:
from langchain.tools import ToolRuntime
import os
import requests
from dotenv import load_dotenv
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langgraph.config import get_stream_writer

# 1. 加载环境变量
load_dotenv()

AMAP_KEY = os.getenv("AMAP_KEY")

load_dotenv()


@tool
def get_weather(city: str, runtime: ToolRuntime) -> str:
  """
  使用高德地图 Web API 查询中国城市的实时天气。
  参数:
      city: 城市名称（中文，如 "北京"、"深圳"）。
  返回:
      城市的实时天气描述，包括天气现象、温度、风力等。
  """

  writer = get_stream_writer()

  writer(f"正在查询 {city} 的天气数据...")

  url = f"https://restapi.amap.com/v3/weather/weatherInfo?key={AMAP_KEY}&city={city}&extensions=base&output=JSON"

  resp = requests.get(url).json()

  writer(f"已获取 {city} 的天气数据。")

  if resp.get("status") != "1":
    return f"无法查询到 {city} 的天气信息。错误信息: {resp}"

  lives = resp.get("lives", [])
  if not lives:
    return f"没有找到 {city} 的天气数据。"

  weather_info = lives[0]
  weather = weather_info["weather"]
  temperature = weather_info["temperature"]
  wind = weather_info["winddirection"]
  wind_power = weather_info["windpower"]
  humidity = weather_info["humidity"]
  report_time = weather_info["reporttime"]

  writer(
    f"{city} 当前天气：{weather}，气温 {temperature}°C，"
    f"风向 {wind}，风力 {wind_power} 级，湿度 {humidity}%。"
    f"（数据更新时间：{report_time}）"
  )

  return (
    f"{city} 当前天气：{weather}，气温 {temperature}°C，"
    f"风向 {wind}，风力 {wind_power} 级，湿度 {humidity}%。"
    f"（数据更新时间：{report_time}）"
  )


# 创建 LangChain 代理
agent = create_agent(
  model="gpt-4o-mini",
  tools=[get_weather],
)

# 调用代理并打印流式结果
for mode, chunk in agent.stream(
  {"messages": [{"role": "user", "content": "Shenzhen 的天气如何？"}]},
  stream_mode=[
    "custom",
    # "updates", 
    # "values"
  ],  # 多种模式一起开
):
  print(f"Stream mode: {mode}, chunk: {chunk}")
  # if mode == "custom":
  #     # 直接是你 writer(...) 写入的内容（字符串或任意可序列化对象）
  #     print("[custom]", chunk)
  # elif mode == "updates":
  #     # 每一步图节点更新（包含节点名、增量等）
  #     print("[updates]", chunk)
  # elif mode == "values":
  #     # 这是“完整状态字典”，所以要用下标访问，不是 .message
  #     # messages 通常是列表，元素为 dict 或 Message 对象，以下兼容两种写法
  #     messages = chunk.get("messages", [])
  #     if messages:
  #         last = messages[-1]
  #         content = last["content"] if isinstance(last, dict) else last.content
  #         role = last.get("role") if isinstance(last, dict) else getattr(last, "role", None)
  #         print("[values] last role:", role, "content:", content)


Stream mode: custom, chunk: 正在查询 深圳 的天气数据...
Stream mode: custom, chunk: 已获取 深圳 的天气数据。
Stream mode: custom, chunk: 深圳 当前天气：阴，气温 24°C，风向 北，风力 ≤3 级，湿度 79%。（数据更新时间：2025-10-30 23:30:46）
